In [1]:
# !pip install "opencv-python-headless<4.3"

  Using cached opencv_python_headless-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl (44.3 MB)
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [2]:
import numpy as np
from keras.models import load_model
import cv2
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
# from keras.preprocessing.image import img_to_array
from time import sleep
import time

In [3]:
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
classifier =load_model(r'model.h5')


emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

cap = cv2.VideoCapture(0)
now = time.time()
future = now + 10
detected_emotions = []

while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces = face_classifier.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)



        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            label=emotion_labels[prediction.argmax()]
            label_position = (x,y-10)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            detected_emotions.append(label)
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF== ord('q'):
        break

    if time.time() > future:  ##after 10 second music will play
            cv2.destroyAllWindows()
            print("Music Player Opens", time)
            future = time.time() + 10
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 174ms/step


2022-11-11 18:22:24.599740: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 12ms/step
Music Player Opens <module 'time' (built-in)>
1/1 [==============================] - 0s 12ms/step


In [4]:
def avr_emotion(detected_emotions):

    emotion_dict = { 
        'Fear': 0,
        'Neutral':0,
        'Happy':0,
        'Sad':0,
        'Disgust':0,
        'Angry':0,
        'Surprised':0
    }

    for emotion in detected_emotions:
        if emotion == 'Neutral':
            emotion_dict['Neutral'] +=1
        elif emotion == 'Fear':
            emotion_dict['Fear'] +=1
        elif emotion =='Sad':
            emotion_dict['Sad'] +=1
        elif emotion == 'Happy':
            emotion_dict['Happy'] +=1
        elif emotion =='Disgust':
            emotion_dict['Disgust'] +=1
        elif emotion == 'Angry':
            emotion_dict['Angry'] +=1
        elif emotion =='Surprise':
            emotion_dict['Surprised'] +=1
    return(emotion_dict)

In [5]:
emo=avr_emotion(detected_emotions)
print(emo)

{'Fear': 1, 'Neutral': 5, 'Happy': 70, 'Sad': 0, 'Disgust': 0, 'Angry': 0, 'Surprised': 14}


In [7]:
strongest_emo= max(emo, key=emo.get)
print(strongest_emo)

Happy


: 